# Imports

In [20]:
import pandas as pd
import numpy as np
import spreg
import libpysal
import geopandas as gpd
from libpysal.weights import Queen
from spreg import ML_Lag

# Data

In [21]:
# Read in the data
# db = libpysal.io.open("../_data/shapefile/map_municipalities.dbf","r")
db = pd.read_csv('../_data/data_municipalities_final.csv')

# Train and test datasets (test on Wielkopolskie)
db["municipality_code_str"] = db["municipality_code"].astype(str)
db.loc[:607, 'municipality_code_str'] = '0' + db.loc[:607, 'municipality_code'].astype(str)
db_train = db[db["municipality_code_str"].str[0:2] != "30"]
db_test = db[db["municipality_code_str"].str[0:2] == "30"]
# drop
db_train.drop(columns=["municipality_code_str"], inplace=True)
db_test.drop(columns=["municipality_code_str"], inplace=True)
print(db_train.shape, db_test.shape)
# Specify the path to the shapefile
shapefile_path = '../_data/shapefile/map_municipalities.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Train and test datasets
gdf_train = gdf[gdf["mncplty_c"].str[0:2] != "30"]
gdf_test = gdf[gdf["mncplty_c"].str[0:2] == "30"]
print(gdf_train.shape, gdf_test.shape)

/tmp/ipykernel_282107/433270946.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_train.drop(columns=["municipality_code_str"], inplace=True)
/tmp/ipykernel_282107/433270946.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_test.drop(columns=["municipality_code_str"], inplace=True)


(2251, 49) (226, 49)
(2251, 3) (226, 3)


# SAR Model

## On train dataset

In [22]:
# Create a spatial weights object
w = Queen.from_dataframe(gdf_train)
ds_name = "municipalities_train"
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
y.shape = (len(y),1)
x_names = db_train.columns.to_list()
x_names.remove("county_code")
x_names.remove("county_name")
x_names.remove("voivodeship")
x_names.remove("municipality_code_old")
x_names.remove("municipality_code")
x_names.remove("splag_percent_vaccinated")
x_names.remove("percent_vaccinated")
x_names.remove("percent_vaccinated_log")
x_names.remove("municipality_name")
x = db_train[x_names].values.astype('float64')
w_name = "border_contiguity"
w.transform = 'r'
mllag = ML_Lag(y=y,x=x,w=w,name_y=y_name,name_x=x_names,name_w=w_name,name_ds=ds_name,vm=True, latex=True)

/tmp/ipykernel_282107/3028427859.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_train)


In [23]:
print(mllag.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :municipalities_train
Weights matrix      :border_contiguity
Dependent Variable  :percent_vaccinated_log                Number of Observations:        2251
Mean dependent var  :      0.0650                Number of Variables   :          42
S.D. dependent var  :      0.3194                Degrees of Freedom    :        2209
Pseudo R-squared    :      0.0179
Spatial Pseudo R-squared:  0.0159
Log likelihood      :  -4806.3278
Sigma-square ML     :      3.0582                Akaike info criterion :    9696.656
S.E of regression   :      1.7488                Schwarz criterion     :    9936.859

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
----------------------------------------

### RMSE in-sample

In [24]:
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
mllag.predy
rmse = np.sqrt(np.sum((y - mllag.predy)**2)/len(y))
print(rmse)

1.7487635476473689


## On test dataset

In [25]:
# firguring out betas and rho
# first beta is intercept
# last beta is rho
mllag.betas, mllag.rho

(array([[-3.46721754e+01],
        [-1.98181847e+07],
        [ 1.98181847e+07],
        [ 1.98181847e+07],
        [ 2.23403325e-03],
        [-5.36711039e-04],
        [ 9.12000787e-03],
        [-7.08041878e-07],
        [-2.20152232e-05],
        [-7.08957573e-03],
        [-3.11691898e-03],
        [-8.57309179e-06],
        [ 1.04768107e-03],
        [-3.17522809e-04],
        [ 1.89313899e+01],
        [-9.45738749e+00],
        [-9.09819112e+00],
        [ 5.95233636e-05],
        [-1.95080128e-04],
        [-4.80013029e-05],
        [-2.16734104e-03],
        [ 8.57773315e-03],
        [ 3.15582612e-03],
        [ 1.51778169e-01],
        [ 8.95963134e-02],
        [ 1.40411696e-01],
        [ 1.47515165e-01],
        [-3.67181526e-05],
        [ 2.46734645e-01],
        [ 2.50964296e-01],
        [ 2.84131509e-01],
        [ 1.80715019e-03],
        [-5.51296231e-02],
        [-5.82266870e-04],
        [ 4.42496436e-03],
        [-1.06635243e-02],
        [ 3.91338094e-03],
 

In [26]:
# Predict on the test dataset
# Prepare y_test
y_test = db_test["percent_vaccinated_log"].values
# Create a spatial weights object
w = Queen.from_dataframe(gdf_test)
# Calcualte Wy
w_array = w.to_sparse().toarray()
row_sums = w_array.sum(axis=1)
w_array_normalized = w_array / row_sums[:, np.newaxis]
Wy = np.dot(w_array_normalized, y_test)
# Calculate y_pred
y_pred = mllag.betas[0] + np.dot(db_test[x_names].values,mllag.betas[1:-1]) + mllag.betas[-1]*Wy
# Calculate RMSE
rmse = np.sqrt(np.sum((y_test - y_pred)**2)/len(y_test))
print(rmse)

15.682449430552015


/tmp/ipykernel_282107/895585023.py:5: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_test)


# SDM

## In-sample

In [28]:
# Create a spatial weights object
w = Queen.from_dataframe(gdf_train)
ds_name = "municipalities_train"
y_name = "percent_vaccinated_log"
y = db_train[y_name].values
y.shape = (len(y),1)
x_names = db_train.columns.to_list()
x_names.remove("county_code")
x_names.remove("county_name")
x_names.remove("voivodeship")
x_names.remove("municipality_code_old")
x_names.remove("municipality_code")
x_names.remove("splag_percent_vaccinated")
x_names.remove("percent_vaccinated")
x_names.remove("percent_vaccinated_log")
x_names.remove("municipality_name")
x = db_train[x_names].values.astype('float64')
w_name = "border_contiguity"
w.transform = 'r'
mllag_SDM = ML_Lag(y,x,w,slx_lags=1,name_y=y_name,name_x=x_names,name_w=w_name,name_ds=ds_name,vm=True, latex=True) 


/tmp/ipykernel_282107/3696663853.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_train)


/home/maciej/anaconda3/envs/mlenv/lib/python3.8/site-packages/spreg/diagnostics.py:354: RuntimeWarning: invalid value encountered in sqrt
  se_result = np.sqrt(variance)
/home/maciej/anaconda3/envs/mlenv/lib/python3.8/site-packages/spreg/diagnostics.py:170: RuntimeWarning: invalid value encountered in sqrt
  ) / np.sqrt(variance)


In [29]:
print(mllag_SDM.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG WITH SLX - SPATIAL DURBIN MODEL (METHOD = FULL)
-------------------------------------------------------------------------------------------------
Data set            :municipalities_train
Weights matrix      :border_contiguity
Dependent Variable  :percent_vaccinated_log                Number of Observations:        2251
Mean dependent var  :      0.0650                Number of Variables   :          82
S.D. dependent var  :      0.3194                Degrees of Freedom    :        2169
Pseudo R-squared    :      0.1150
Spatial Pseudo R-squared:  0.0771
Log likelihood      :  -3919.8591
Sigma-square ML     :      1.7567                Akaike info criterion :    8003.718
S.E of regression   :      1.3254                Schwarz criterion     :    8472.687

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-St